In [4]:
import sys
!{sys.executable} -m pip install ultralytics opencv-python numpy


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import os
from functools import cmp_to_key

In [21]:

MODEL_PATH = "./best.pt"
RGB_DIR = "./Public_data_task_3/Public_data/Public_data_train/rgb"
DEPTH_DIR = "./Public_data_task_3/Public_data/Public_data_train/depth"

VISUALIZATION_SAVE_DIR = "./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_visualization"
CROP_SAVE_DIR = "./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs"
os.makedirs(VISUALIZATION_SAVE_DIR, exist_ok=True)
os.makedirs(CROP_SAVE_DIR, exist_ok=True)


DEPTH_SCALE = 0.001 # Hệ số chuyển đổi giá trị depth sang mét

# Cấu hình logic xử lý và lựa chọn
FIXED_ROI_XYWH = (560, 150, 300, 330) # Tọa độ và kích thước của "Vùng Quan Tâm" (Region of Interest) cố định. Chỉ các bưu kiện có tâm nằm trong vùng này mới được xét.
HEIGHT_TOLERANCE_METERS = 0.005 # Ngưỡng chênh lệch chiều cao (5mm). Nếu hai bưu kiện chênh nhau ít hơn giá trị này, chúng được coi là cao bằng nhau.
MIN_MASK_PIXEL = 500 # Diện tích tối thiểu (tính bằng pixel) để một đối tượng được coi là một bưu kiện hợp lệ, giúp loại bỏ nhiễu.

# Các hàm

# Kiểm tra xem một điểm tâm (cx, cy) có nằm trong vùng ROI đã định nghĩa hay không.
def is_center_in_roi(bbox_center, roi_xywh):
    cx, cy = bbox_center
    x, y, w, h = roi_xywh
    return (x <= cx < x + w) and (y <= cy < y + h)

# Hàm đơn giản hóa, chỉ lấy giá trị pixel tại một điểm trên ảnh depth, kiểm tra xem nó có hợp lệ không (khác 0) và đổi giá trị đó ra mét bằng cách nhân với DEPTH_SCALE.
def get_depth_in_meters(cx, cy, depth_map, depth_scale):
    if cx < 0 or cy < 0 or cy >= depth_map.shape[0] or cx >= depth_map.shape[1]:
        return None
    z_val = depth_map[int(cy), int(cx)]
    if z_val == 0:
        return None
    return z_val * depth_scale

# Load ảnh
def load_rgb_depth(rgb_path, depth_path):
    rgb = cv2.imread(rgb_path)
    depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)
    if rgb is None: raise FileNotFoundError(f"RGB image not found: {rgb_path}")
    if depth is None: raise FileNotFoundError(f"Depth image not found: {depth_path}")
    if len(depth.shape) == 3: depth = depth[:, :, 0]
    return rgb, depth

# Thay đổi kích thước mask theo hình dạng của hình ảnh mục tiêu.
def mask_resize_to_image(mask, target_shape):
    target_h, target_w = target_shape[0], target_shape[1]
    resized = cv2.resize(mask.astype(np.float32), (target_w, target_h), interpolation=cv2.INTER_NEAREST)
    return (resized > 0.5).astype(np.uint8)

# Tìm bounding box từ mặt nạ nhị phân.
def bbox_from_mask(mask):
    cnts, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not cnts: return None
    c = max(cnts, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(c)
    return (x, y, w, h) if w > 0 and h > 0 else None

# Tính toán tâm của bounding box
def center_of_bbox(bbox):
    x, y, w, h = bbox
    return (x + w // 2, y + h // 2)

# Tìm kiếm dải màu HSV của băng tải một cách thích ứng bằng cách lấy mẫu các góc hình ảnh
def get_adaptive_hsv_range(rgb_image):
    h, w, _ = rgb_image.shape
    sample_size = 20
    corners = [rgb_image[0:sample_size, 0:sample_size], rgb_image[0:sample_size, w-sample_size:w], rgb_image[h-sample_size:h, 0:sample_size], rgb_image[h-sample_size:h, w-sample_size:w]]
    hsv_samples = [cv2.cvtColor(corner, cv2.COLOR_BGR2HSV) for corner in corners]
    h_vals = np.concatenate([c[:, :, 0].flatten() for c in hsv_samples])
    s_vals = np.concatenate([c[:, :, 1].flatten() for c in hsv_samples])
    v_vals = np.concatenate([c[:, :, 2].flatten() for c in hsv_samples])
    if h_vals.size == 0: return None, None
    median_h, median_s, median_v = np.median(h_vals), np.median(s_vals), np.median(v_vals)
    h_tolerance, s_tolerance, v_tolerance = 15, 80, 80
    lower_bound = np.array([max(0, median_h - h_tolerance), max(0, median_s - s_tolerance), max(0, median_v - v_tolerance)])
    upper_bound = np.array([min(179, median_h + h_tolerance), min(255, median_s + s_tolerance), min(255, median_v + v_tolerance)])
    return lower_bound, upper_bound

# Giải pháp dự phòng: Phân đoạn các đối tượng bằng cách tìm mọi thứ KHÔNG phải là màu của băng tải
def color_based_fallback(rgb):
    hsv_image = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
    lower_conveyor, upper_conveyor = get_adaptive_hsv_range(rgb)
    if lower_conveyor is None: return []
    conveyor_mask = cv2.inRange(hsv_image, lower_conveyor, upper_conveyor)
    object_mask = cv2.bitwise_not(conveyor_mask)
    kernel = np.ones((5,5), np.uint8)
    object_mask = cv2.morphologyEx(object_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    object_mask = cv2.morphologyEx(object_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    cnts, _ = cv2.findContours(object_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidate_masks = []
    H, W = rgb.shape[:2]
    for c in cnts:
        if cv2.contourArea(c) >= MIN_MASK_PIXEL:
            mask = np.zeros((H, W), dtype=np.uint8)
            cv2.drawContours(mask, [c], -1, color=1, thickness=-1)
            candidate_masks.append(mask)
    return candidate_masks

# So sánh để chọn bưu kiện
def compare_parcels_simplified(p1, p2):
    """Hàm so sánh đã được cập nhật để chỉ dùng độ sâu."""
    z1, z2 = p1['depth_meters'], p2['depth_meters']
    if abs(z1 - z2) > HEIGHT_TOLERANCE_METERS:
        return -1 if z1 < z2 else 1
    cy1, cy2 = p1['center'][1], p2['center'][1]
    return -1 if cy1 > cy2 else 1

# Tải mô hình
print("Loading YOLO model...")
model = YOLO(MODEL_PATH)
print("Model loaded successfully.")

# Vòng lặp chính
for rgb_name in sorted(os.listdir(RGB_DIR)):
    if not rgb_name.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    base_name = os.path.splitext(rgb_name)[0]
    rgb_path = os.path.join(RGB_DIR, rgb_name)
    depth_path = os.path.join(DEPTH_DIR, rgb_name)
    try:
        rgb, depth = load_rgb_depth(rgb_path, depth_path)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        continue

    # 1) Nhận tất cả các mask bưu kiện từ YOLO
    results = model.predict(rgb, verbose=False)
    valid_masks = []
    if results and results[0].masks is not None:
        raw_masks = results[0].masks.data.cpu().numpy()
        for raw in raw_masks:
            m = mask_resize_to_image(raw, rgb.shape)
            if np.count_nonzero(m) >= MIN_MASK_PIXEL:
                valid_masks.append(m)

    # 2) Nếu YOLO không tìm thấy gì, sử dụng phương pháp dự phòng 
    if not valid_masks:
        print(f"YOLO found no objects in {rgb_name}. Using robust color-based fallback...")
        valid_masks = color_based_fallback(rgb)
    if not valid_masks:
        print(f"No candidate objects found for {rgb_name}")
        continue

    # 3) Xây dựng danh sách bưu kiện chỉ với depth
    all_candidates = []
    for mask in valid_masks:
        bbox = bbox_from_mask(mask)
        if bbox is None:
            continue
        cx, cy = center_of_bbox(bbox)
        z_meters = get_depth_in_meters(cx, cy, depth, DEPTH_SCALE)
        if z_meters is not None:
            all_candidates.append({
                "mask": mask, #Thêm mask vào để cắt sau này
                "bbox": bbox,
                "center": (cx, cy),
                "depth_meters": z_meters
            })

    # 4) Lọc ứng viên trong ROI
    roi_candidates = [c for c in all_candidates if is_center_in_roi(c['center'], FIXED_ROI_XYWH)]

    if not roi_candidates:
        print(f"No parcels found inside the ROI for {rgb_name}")
        continue

    # 5) Sắp xếp và tìm ứng viên tốt nhất
    roi_candidates.sort(key=cmp_to_key(compare_parcels_simplified))
    best = roi_candidates[0]
    
    # 6) Tạo ảnh trực quan hóa
    x, y, w, h = best["bbox"]
    cx, cy = best["center"]
    Z = best["depth_meters"]
    
    vis_img = rgb.copy()
    cv2.rectangle(vis_img, (x, y), (x + w, y + h), color=(0, 255, 0), thickness=2)
    roi_x, roi_y, roi_w, roi_h = FIXED_ROI_XYWH
    cv2.rectangle(vis_img, (roi_x, roi_y), (roi_x + roi_w, roi_y + roi_h), color=(255, 0, 0), thickness=2)
    
    text = f"Center:({cx},{cy}) Depth Z:{Z:.2f}m"
    ty = y - 10 if y - 10 > 10 else y + h + 25
    cv2.putText(vis_img, text, (x, ty), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    
    vis_out_name = os.path.join(VISUALIZATION_SAVE_DIR, f"{base_name}_visualization.png")
    cv2.imwrite(vis_out_name, vis_img)
    
    print(f"Processed {rgb_name} -> Selected Parcel Center:({cx},{cy}), Depth: Z={Z:.4f} meters")

    # 7) Cắt và lưu ảnh cho Giai đoạn 2 
    top_mask = best["mask"]
    
    rgb_crop = rgb[y:y+h, x:x+w]
    depth_crop = depth[y:y+h, x:x+w]
    mask_crop = top_mask[y:y+h, x:x+w]
    
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_rgb_crop.png"), rgb_crop)
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_depth_crop.png"), depth_crop)
    cv2.imwrite(os.path.join(CROP_SAVE_DIR, f"{base_name}_mask_crop.png"), mask_crop * 255)
    print(f" -> Saved cropped outputs for {base_name} to {CROP_SAVE_DIR}")


print("\nFinished processing all images.")



Loading YOLO model...
Model loaded successfully.
Processed 0000.png -> Selected Parcel Center:(616,386), Depth: Z=1.0640 meters
 -> Saved cropped outputs for 0000 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0001.png -> Selected Parcel Center:(620,386), Depth: Z=1.0640 meters
 -> Saved cropped outputs for 0001 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0002.png -> Selected Parcel Center:(596,285), Depth: Z=1.0500 meters
 -> Saved cropped outputs for 0002 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
No parcels found inside the ROI for 0003.png
No parcels found inside the ROI for 0004.png
Processed 0005.png -> Selected Parcel Center:(628,368), Depth: Z=1.0600 meters
 -> Saved cropped outputs for 0005 to ./Public_data_task_3/Public_data/Public_data_train/topmost_simplified_cropped_outputs
Processed 0006.png -> Selected Parcel Center:(628